U-Net，图像分割呢是输入一张图片，输出一个mask（在这里就是0 1两位啦，有人的地方是1，其他地方是0，如果要在一个图片分出不同种类就会有1234这样）

我要把Dataset改成label是图片的形式（在我用的数据集指annotation里的黑白图）

In [ ]:
#针对U-Net这个情况我想到的是这个解决方案！
#这是网络的
import torchvision.transforms.functional as tf
from PIL import Image

def my_transform1(image, mask):

        # 拿到角度的随机数。angle是一个-180到180之间的一个数
        angle = transforms.RandomRotation.get_params([-180, 180])
        # 对image和mask做相同的旋转操作，保证他们都旋转angle角度
        image = image.rotate(angle)
        mask = mask.rotate(angle)
        
        image = tf.to_tensor(image)
        mask = tf.to_tensor(mask)
        return image, mask

def my_transform2(image, mask):
    # 50%的概率应用垂直，水平翻转。
    if random.random() > 0.5:
        image = tf.hflip(image)
        mask = tf.hflip(mask)
    if random.random() > 0.5:
        image = tf.vflip(image)
        mask = tf.vflip(mask)
    image = tf.to_tensor(image)
    mask = tf.to_tensor(mask)
    return image, mask
————————————————
版权声明：本文为CSDN博主「Onwaier」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/happyeveryday62/java/article/details/104350332

In [ ]:
#为了可以同时处理image和mask，设置了一下的函数
def my_transform1(image, mask):
    #my_transform1是针对训练集的，可以做到data augmentation的效果
    if random.random() > 0.5:
        image = tf.hflip(image)
        mask = tf.hflip(mask)
    if random.random() > 0.5:
        image = tf.vflip(image)
        mask = tf.vflip(mask)
    #对image进行resize，totensor，还有normalize
    transform_image = T.Compose([   
        T.Resize([256,256]),        
        T.ToTensor(),
        T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    image = transform_image(image)
    #对mask进行resize，把它转换成array，把所有大于1的地方换成1，最后转成tensor。
    mask=T.functional.resize(mask,(256,256))
    mask=np.array(mask)
    mask=(mask>=1).astype(int)
    mask = T.functional.to_tensor(mask).float()
    return image, mask
    
def my_transform2(image, mask):
    #my_transform2是针对valid&test的，所以就不需要rotation之类的处理了。
    #对image进行resize，totensor，还有normalize
    transform_image = T.Compose([
        T.Resize([256, 256]),
        T.ToTensor(),
        T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    image = transform_image(image)
    #对mask进行resize，把它转换成array，把所有大于1的地方换成1，最后转成tensor。
    mask=T.functional.resize(mask,(256,256))
    mask=np.array(mask)
    mask=(mask>=1).astype(int)
    mask = T.functional.to_tensor(mask).float()
    return image, mask

初始MyDataset

In [ ]:

# encoding: utf-8
"""
@author: Ming Cheng
@contact: ming.cheng@dukekunshan.edu.cn
"""

import os
from PIL import Image
from torch.utils import data
from torchvision import transforms as T


transform_train = T.Compose([   
        T.Resize([256, 256]),
        T.RandomRotation(degrees=45, expand=True),
        T.CenterCrop([224,224]),
        T.RandomHorizontalFlip(p=0.5),
        T.RandomVerticalFlip(p=0.5),
        T.ToTensor(),
        T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        

transform_test = T.Compose([
        T.Resize([224, 224]),
        T.ToTensor(),
        T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])  
        

class MyDataset(data.Dataset):
    
    
    def __init__(self, file_path=None, classes=None, transform=None):   
        """
        初始化自定义Dataset类的参数
        Attributes
            file_path: 字符串，数据集的存储路径，例如‘./UCF101/train’ 或 './UCF101/eval'等
            classes  : 列表，每个元素为一个字符串，代表一个子类别，例如['dog', 'airplane', ...]等
            transform: 传入一个从torchvision.transforms定义的数据预处理
        """
        self.classes = classes
        self.transform = transform
        # 初始化给定文件夹下的所有数据
        self.init_all_data(file_path) 

        return None
        

    def init_all_data(self, file_path):
        """
        初始化该数据集内所有的图像及其对应的标签，保存在self.images和self.labels两个列表内
        Attributes
            file_path: 字符串，数据集文件夹的存储路径
        """
        # 初始化两个列表，记录该数据集内每一张图片的完整路径及其对应的标签
        self.images = []
        self.labels = []
        # 遍历所有的子类别，并得到每个子类别对应的文件夹路径
        for idx, cls in enumerate(self.classes):
            cls_path = os.path.join(file_path, cls)
            # 遍历当前子类文件夹下的所有图片
            for img in os.listdir(cls_path):
                # 得当当前图片的完整路径，若是有效图片，则记录该图片的完整路径及其标签
                img = os.path.join(cls_path, img)
                if self.is_valid_image(img):
                    self.images.append(img)
                    self.labels.append(idx)

        return None

        
    def is_valid_image(self, img_path):
        """
        判断图片是否为可以打开的有效文件
        Attributes
            img_path: 字符串，待检测图片的存储路径
        Returns
            valid: 布尔变量，True/False分别表示该图片是否可以正常打开
        """
        try:
            # 若读取成功，设valid为True
            i = Image.open(img_path)
            valid = True
        except:
            # 若读取失败，设valid为False
            valid = False
            
        return valid
        

    def __getitem__(self, idx):
        """
        按给定索引，获取对应的图片及其标签
        Attributes
            idx: int类型数字，表示目标图像的索引
        Returns
            image: 一个打开的PIL.Image对象，是PIL库存储图像的一种数据格式（类似于OpenCV利用numpy张量存储图像）
            label: int类型，表示对应的类别，例如假设self.classes=['cat', 'dog', 'airplane']，则label=1代表‘dog'类别；
                   对于pytorch的分类，不需要特意将其变成onehot向量，因为crossentropy函数内置了这部分功能。
        """
        # 利用PIL.Image.open打开图片，并将其强制转化为RGB格式（防止数据集中混杂灰度图，导致读取出单通道图片，送入网络因矩阵维度不一致而报错）
        image = Image.open(self.images[idx]).convert('RGB')
        # 获取对应的标签
        label = self.labels[idx]
        # 进行预处理的变换
        if self.transform:
            image = self.transform(image)
              
        return image, label
   

    def __len__(self):
        """
        获取数据集中图像的总数，该方法的作用是用于DataLoader去调用，从而获取在给定Batch Size的情况下，一个Epoch的总长，
        从而可以在一个Epoch结束时实现shuffle数据集的功能
        """

        return len(self.images)

现在进行了改造！ 

1.label这里是图片，所以增加了输入mask的path！

2.由于数据包中test的mask缺失，决定把val的一半作为test。

3.使用txt来进行文件的导入

4.使用之前的my_transformation 来同时处理image和label。

In [ ]:
class MyDataset(data.Dataset):
    
    
    def __init__(self, file_path=None, mask_path=None, transform=None,data="train"):   
        """
        初始化自定义Dataset类的参数
        Attributes
            file_path: 字符串，数据集的存储路径，例如‘./UCF101/train’ 或 './UCF101/eval'等
            mask_path: 字符串，数据集的存储路径，例如‘./UCF101/train_annotation’等
            transform: 传入一个从torchvision.transforms定义的数据预处理
        """
        self.count=0
        self.transform = transform
        if data=='train':
            self.name=open(os.path.join("../input/singleperson/TrainVal_images","train_id.txt"))
        elif data=="valid":
            self.name=open(os.path.join("../input/singleperson/TrainVal_images","val_id.txt"))
            #我要改变一下，就是这里的val_id会被分出去一些,就是10000的一半！
        elif data=="test":
            self.name=open(os.path.join("../input/singleperson/TrainVal_images","val_id.txt"))
        # 初始化给定文件夹下的所有数据
        self.init_all_data(file_path,mask_path,data) 

        return None
        

    def init_all_data(self,file_path,mask_path,data):
        """
        初始化该数据集内所有的图像及其对应的标签，保存在self.images和self.labels两个列表内
        Attributes
            file_path: 字符串，数据集文件夹的存储路径
            mask_path: 字符串，数据集文件夹的存储路径
        """
        # 初始化两个列表，记录该数据集内每一张图片的完整路径及其对应的mask
        self.images = []
        self.labels = []
        for name in self.name:
            # 得当当前图片的完整路径，若是有效图片，则记录该图片
            self.count+=1
            if data=="train"and self.count<=2000:
                img = os.path.join(file_path,name[0:-1]+".jpg")
                mask = os.path.join(mask_path,name[0:-1]+".png")
                if self.is_valid_image(img) and self.is_valid_image(mask):
                    self.images.append(img)
                    self.labels.append(mask)
            if data=="valid" and self.count<=600:
                img = os.path.join(file_path,name[0:-1]+".jpg")
                mask = os.path.join(mask_path,name[0:-1]+".png")
                if self.is_valid_image(img) and self.is_valid_image(mask):
                    self.images.append(img)
                    self.labels.append(mask)
            elif data=="test" and self.count>=5000:
                img = os.path.join(file_path,name[0:-1]+".jpg")
                mask = os.path.join(mask_path,name[0:-1]+".png")
                if self.is_valid_image(img) and self.is_valid_image(mask):
                    self.images.append(img)
                    self.labels.append(mask)
        return None

        
    def is_valid_image(self, img_path):
        """
        判断图片是否为可以打开的有效文件
        Attributes
            img_path: 字符串，待检测图片的存储路径
        Returns
            valid: 布尔变量，True/False分别表示该图片是否可以正常打开
        """
        try:
            # 若读取成功，设valid为True
            i = Image.open(img_path)
            valid = True
        except:
            # 若读取失败，设valid为False
            valid = False
            
        return valid
        

    def __getitem__(self, idx):
        """
        按给定索引，获取对应的图片及其标签
        Attributes
            idx: int类型数字，表示目标图像的索引
        Returns
            image: 一个打开的PIL.Image对象，是PIL库存储图像的一种数据格式（类似于OpenCV利用numpy张量存储图像）
            label: Image类型，表示对应的mask
        """
        # 利用PIL.Image.open打开图片，并将其强制转化为RGB格式（防止数据集中混杂灰度图，导致读取出单通道图片，送入网络因矩阵维度不一致而报错）
        image = Image.open(self.images[idx]).convert('RGB')
        # 获取对应的mask
        label = Image.open(self.labels[idx])
        #获取mask后就要把它转换成全0，1的array，再换成Image
        
        # 进行预处理的变换
        if self.transform:
            image,label = self.transform(image,label)
        return image, label
   

    def __len__(self):
        """
        获取数据集中图像的总数，该方法的作用是用于DataLoader去调用，从而获取在给定Batch Size的情况下，一个Epoch的总长，
        从而可以在一个Epoch结束时实现shuffle数据集的功能
        """

        return len(self.images)

失败品，但可以学习如何让图片看起来像黑白图

In [ ]:
label = Image.open(self.labels[idx])
#获取mask后就要把它转换成全0，1的array，再换成Image
label=np.array(label)
label=(label>0).astype(int)
label=Image.fromarray(np.uint8(label))

In [ ]:
#另一种处理方式
threshold=1
table = []
for i in range(256):
    if i < threshold:
        table.append(0)
    else:
        table.append(1)
label = Image.open(self.labels[idx]).point(table, '1')

现在使用的是在my_transformation 里将图片进行转换。